# 找出图像所在的位置

In [1]:
import os
import cv2
file_name1="left.png"
abs_left_path=os.path.join(os.path.abspath('.'),file_name1)
print(abs_left_path)
file_name2="right.png"
abs_right_path=os.path.join(os.path.abspath('.'),file_name2)
print(abs_right_path)

C:\Users\32995\Music\homework\left.png
C:\Users\32995\Music\homework\right.png


In [5]:
img1=cv2.imread(abs_left_path)
img2=cv2.imread(abs_right_path)
cv2.imshow('image',img1)
cv2.imshow('image',img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 检测图像的关键特征点

In [3]:
import cv2
import numpy as np



def sift_keypoints_detect(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    keypoints, features = sift.detectAndCompute(image, None)
    keypoints_image = cv2.drawKeypoints(gray_image, keypoints, None,
                                        flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)
    return keypoints_image, keypoints, features




# 使用KNN检测来自左右图像的SIFT特征进行匹配

In [ ]:

def get_feature_point_ensemble(features_right, features_left):
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(features_right, features_left, k=2)
    matches = sorted(matches, key=lambda x: x[0].distance / x[1].distance)
    good = []
    for m, n in matches:
        ratio = 0.6
        if m.distance < ratio * n.distance:
            good.append(m)
    return good





# 计算视角变换矩阵H，用H对右图进行变换并返回全景拼接图像

In [ ]:

def Panorama_stitching(image_right, image_left):
    _, keypoints_right, features_right = sift_keypoints_detect(image_right)
    _, keypoints_left, features_left = sift_keypoints_detect(image_left)
    goodMatch = get_feature_point_ensemble(features_right, features_left)

    if len(goodMatch) > 4:
        Point_coordinates_right = np.float32([keypoints_right[m.queryIdx].pt for m in goodMatch]).reshape(-1, 1, 2)
        Point_coordinates_left = np.float32([keypoints_left[m.trainIdx].pt for m in goodMatch]).reshape(-1, 1, 2)
        ransacReprojThreshold = 4
        Homography, status = cv2.findHomography(Point_coordinates_right, Point_coordinates_left, cv2.RANSAC,
                                                ransacReprojThreshold)

        Panorama = cv2.warpPerspective(image_right, Homography,
                                       (image_right.shape[1] + image_left.shape[1], image_right.shape[0]))
        Panorama[0:image_left.shape[0], 0:image_left.shape[1]] = image_left
        return Panorama


if __name__ == '__main__':
    image_left = cv2.imread(abs_left_path)
    image_right = cv2.imread(abs_right_path)



   # 调整图像的尺寸，并输出左侧和右侧图像的sift检测到的关键点

In [ ]:
 
    image_right = cv2.resize(image_right, None, fx=2, fy=1.2)
    image_left = cv2.resize(image_left, (image_right.shape[1], image_right.shape[0]))

    keypoints_image_right, keypoints_right, features_right = sift_keypoints_detect(image_right)
    keypoints_image_left, keypoints_left, features_left = sift_keypoints_detect(image_left)

    cv2.imshow("keypoint_left", np.hstack(
        (cv2.resize(image_left, None, fx=0.5, fy=0.5), cv2.resize(keypoints_image_left, None, fx=0.5, fy=0.5))))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    cv2.imshow("keypoint_right", np.hstack(
        (cv2.resize(image_right, None, fx=0.5, fy=0.5), cv2.resize(keypoints_image_right, None, fx=0.5, fy=0.5))))
    cv2.waitKey(0)
    cv2.destroyAllWindows()



# 输出两侧图像sift关键点的匹配

In [ ]:
    goodMatch = get_feature_point_ensemble(features_right, features_left)

    all_goodmatch_image = cv2.drawMatches(image_right, keypoints_right, image_left, keypoints_left, goodMatch, None,
                                          None, None, None, flags=2)
    cv2.imshow("SIFT", cv2.resize(all_goodmatch_image, None, fx=0.6, fy=0.6))
    cv2.waitKey(0)
    cv2.destroyAllWindows()


# 最终输出图像的拼接效果

In [ ]:

    Panorama = Panorama_stitching(image_right, image_left)
    cv2.namedWindow("Panoramic view", cv2.WINDOW_AUTOSIZE)
    cv2.imshow("Panoramic view", cv2.resize(Panorama, None, fx=0.6, fy=0.6))
    cv2.waitKey(0)
    cv2.destroyAllWindows()